In [ ]:
import pandas as pd
import geopandas as gpd
import math
import h3
import numpy as np
import statistics
import ipywidgets
import html

In [ ]:
lons = []
lats = []
names = []
rs = []
for i in range(0, 360, 10):
    lon = 149 + math.cos(math.radians(i))
    lat = -35 + math.sin(math.radians(i))
    name = f'Name_{i%3}'
    lons.append(lon)
    lats.append(lat)
    names.append(name)
    rs.append(sum(lons))

In [ ]:
df = pd.DataFrame({'LON':lons, 'LAT':lats, 'NAME':names, 'R':rs})
gdf = gpd.GeoDataFrame(df, geometry=gpd.points_from_xy(df.LON, df.LAT), crs='EPSG:4326')
gdf

In [ ]:
# gdf.geom_type

In [ ]:
gdf.explore()#column='NAME')

In [ ]:
gdf.plot()

In [ ]:
cell = h3.latlng_to_cell(-35, 149, 4)
h3.cell_to_boundary(cell)

In [ ]:
cells = h3.grid_ring(cell, 1)
cells

In [ ]:
boundaries = [h3.cell_to_boundary(cell) for cell in cells]
boundaries

In [ ]:
poly = h3.cells_to_h3shape(cells)

In [ ]:
shapes = [h3.cells_to_h3shape([cell]) for cell in cells]

In [ ]:
df_hex = gpd.GeoDataFrame({'A':list('abcdef'), 'NAME':[f'Name_{i}' for i in range(6)], 'geometry':shapes}, crs=4326)
# df = df.to_crs(epsg=4326)
print(df_hex.crs)
print(df_hex.total_bounds)
df_hex

In [ ]:
df.explore(column='A')

In [ ]:
import ipyleaflet

In [ ]:
def debug(s):
    with open('D:/tmp/l.log', 'a') as f:
        print(s, file=f)

def table(properties, fields):
    lines = []
    for k, v in properties.items():
        if fields is None or k in fields:
            lines.append(f'<tr><th style="text-align: left">{html.escape(k)}</th><td>:{html.escape(str(v))}</td></tr>')

    return '<table style="padding:0;"><tbody>' + ''.join(lines) + '</tbody></table>'

class Explore:
    def __init__(self, df, lon_lat_cols):
        if isinstance(df, gpd.GeoDataFrame):
            if lon_lat_cols:
                raise ValueError('Cannot specify lon_lat cols with a GeoDataFrame')

            if not df.crs.equals(4326):
                df = df.to_crs(4326)

            self.gdf = df
        else:
            if len(lon_lat_cols)!=2:
                raise ValueError('lon_lat_cols must be two column names')

            lon, lat = lon_lat_cols
            self.gdf = gpd.GeoDataFrame(df, geometry=gpd.points_from_xy(lon, lat), crs=4326)

    def _fit(self, change):
        """Fit the map to the bounds after rendering.
        
        Map.fit_bounds() only works after the map has been rendered. Therefore, we observe
        the bounds change when the map is rendered, then refit the bounds."""
        
        self.m.unobserve(self._fit, 'bounds')
        minx, miny, maxx, maxy = self.gdf.total_bounds
        self.m.fit_bounds([[miny, minx], [maxy, maxx]])

    def _hovering(self, feature, **kwargs):
        t = table(feature['properties'], self.tooltip_fields)
        self.htmlw.value = t
            
    def _clicking(self, *args, **kwargs):
        debug(f'@CLICK {args=} {kwargs.keys()=} {kwargs=}')
        coords = kwargs['coordinates']
        feature = kwargs['feature']
        self.popup.location = coords
        # self.popup.child = ipywidgets.HTML('xyzzy ' + table(feature['properties'], self.popup_fields))
        self.popup.child.value = 'xyzzy ' + table(feature['properties'], self.popup_fields)
        self.popup.open_popup()
            
    def explore(self, *, point_style=None, hover_style=None, tooltip_fields=None, popup_fields=None):
        if not point_style:
            point_style = {}

        for k, v in (('fillOpacity', 0.5), ('weight', 2), ('radius', 2)):
            if k not in point_style:
                point_style[k] = v

        if not hover_style:
            hover_style = {}

        for k, v in (('fillOpacity', 0.75),):
            if k not in hover_style:
                hover_style[k] = v

        self.tooltip_fields = tooltip_fields
        self.popup_fields = popup_fields
                
        self.m = ipyleaflet.Map(scroll_wheel_zoom=True)
        geo_data = ipyleaflet.GeoData(
            geo_dataframe=self.gdf,
            point_style=point_style,
            hover_style=hover_style
        )
        geo_data.on_hover(self._hovering)
        self.m.add(geo_data)

        # debug(f'@@POPUP {popup_fields=}')
        if self.popup_fields:
            self.popup = ipyleaflet.Popup(location=(0,0), child=ipywidgets.HTML('popping'), auto_pan=False)
            geo_data.popup = self.popup
            geo_data.on_click(self._clicking)
            self.m.add(self.popup)

        # Always do on-hover tooltips.
        # Note: Actual Leaflet tooltips are after ipyleaflet 0.20.0.
        #
        self.htmlw = ipywidgets.HTML('...')
        control = ipyleaflet.WidgetControl(widget=self.htmlw, position='topright')
        self.m.add(control)

        # Cause a bounds fitting after rendering.
        #
        self.m.observe(self._fit, 'bounds')
    
        return self.m

def explore(df, *, lon_lat_cols=None, point_style=None, tooltip_fields=None, popup_fields=None):
    e = Explore(df, lon_lat_cols)
    return e.explore(point_style=point_style, tooltip_fields=tooltip_fields, popup_fields=popup_fields)

m = explore(gdf, tooltip_fields=['NAME', 'R'], popup_fields=['R', 'NAME'])
m

In [ ]:
explore(df_hex, popup_fields=['R', 'NAME'])

In [ ]:
df_hex.explore()